<a href="https://colab.research.google.com/github/nikbizkit/MMO/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

Загрузим датасет:

In [32]:
df_perf_all = pd.read_csv('vgsales.csv', sep=",")
df_perf_all.head()


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [33]:
df_perf_all.shape

(16598, 11)

Убедимся, что в нашем рабочем датафрейме не будет записей с отсутсвующей видеокартой: 

In [34]:
df_perf_with_Platform = df_perf_all[df_perf_all['Platform'].notnull()]
df_perf_with_Platform = df_perf_with_Platform[~df_perf_with_titulo['Platform'].str.isspace()]

In [35]:
Genre = df_perf_with_Platform['Genre'].values
Genre[0:6]

array(['Sports', 'Platform', 'Racing', 'Sports', 'Role-Playing', 'Puzzle'],
      dtype=object)

In [36]:
Publisher= df_perf_with_Platform['Publisher'].values
Publisher[0:5]

array(['Nintendo', 'Nintendo', 'Nintendo', 'Nintendo', 'Nintendo'],
      dtype=object)

In [37]:
Platform = df_perf_with_Platform['Platform'].values
Platform[0:3]

array(['Wii', 'NES', 'Wii'], dtype=object)

Векторизуем видеокарту с помощью Tf-Idf Vectorizer

In [38]:
tfidfv = TfidfVectorizer()
descr_matrix = tfidfv.fit_transform(Platform)
descr_matrix

<16598x31 sparse matrix of type '<class 'numpy.float64'>'
	with 16598 stored elements in Compressed Sparse Row format>

И с помощью CountVectorizer:

In [39]:
countv = CountVectorizer()
Platform_matrix_co = countv.fit_transform(Platform)
Platform_matrix_co

<16598x31 sparse matrix of type '<class 'numpy.int64'>'
	with 16598 stored elements in Compressed Sparse Row format>

In [40]:
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_names, X_notes, X_descr):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_ids - массив идентификаторов объектов
        X_title - массив названий объектов
        X_overview - массив описаний объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'Genre': pd.Series(X_names, dtype='str'),
            'Publisher': pd.Series(X_notes, dtype='str'),
            'Platform': pd.Series(X_descr, dtype='str'),
            'Dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['Dist'] = dist * scale
            res = self.df.sort_values(by='Dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['Dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['Dist'] = dist * scale
            res = self.df.sort_values(by='Dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['Dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

Выберем тестовый образец, на основе которого мы будем давать рекомендации:

In [41]:
test_name = 7
Genre[test_name]

'Misc'

Зададим его матрицу: 

In [42]:
test_matrix = descr_matrix[test_name]
test_matrix

<1x31 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [43]:
skr1 = SimpleKNNRecommender(descr_matrix, Genre, Publisher, Platform)

In [44]:
test = df_perf_with_Platform.iloc[test_name]
test

Rank                   8
Name            Wii Play
Platform             Wii
Year                2006
Genre               Misc
Publisher       Nintendo
NA_Sales           14.03
EU_Sales             9.2
JP_Sales            2.93
Other_Sales         2.85
Global_Sales       29.02
Name: 7, dtype: object

Делаем рекомендацию на основании видеокарты векторизованного Tf-Idf и косинусного  расстояния:

In [45]:
rec1 = skr1.recommend_for_single_object(15, test_matrix)
rec1

,Genre,Publisher,Platform,Dist
10954,Simulation,Namco Bandai Games,PS4,0.0
10848,Action,Hudson Soft,DS,0.0
10942,Platform,Ubisoft,PS,0.0
10950,Strategy,Electronic Arts,PC,0.0
10964,Misc,Take-Two Interactive,DS,0.0
10949,Sports,Take-Two Interactive,PS3,0.0
10963,Puzzle,Vivendi Games,GBA,0.0
10962,Role-Playing,Sony Computer Entertainment,PS2,0.0
10943,Sports,Electronic Arts,PS3,0.0
10961,Action,Activision,X360,0.0


In [46]:
test_matrix_co = Platform_matrix_co[test_name]
test_matrix_co

<1x31 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [49]:
skr2 = SimpleKNNRecommender(Platform_matrix_co, Genre, Publisher, Platform)

Делаем рекомендации по видеокартам векторизованным CountVectorizer и на основе Евклидова расстояния:

In [50]:
rec2 = skr2.recommend_for_single_object(15, Platform_matrix_co, cos_flag = False)
rec2

,Genre,Publisher,Platform,Dist
10928,Sports,Atari,X360,1.414214e+06
16203,Adventure,Nippon Ichi Software,PSV,1.414214e+06
10934,Adventure,Idea Factory,PSP,1.414214e+06
10933,Strategy,LucasArts,PC,1.414214e+06
10930,Fighting,Konami Digital Entertainment,PS2,1.414214e+06
16202,Action,Idea Factory,DS,1.414214e+06
10929,Platform,Atari,GBA,1.414214e+06
10932,Sports,THQ,PS2,1.414214e+06
11131,Shooter,Atari,GC,1.414214e+06
10925,Role-Playing,Atlus,PS,1.414214e+06
